In [0]:
import os
import numpy as np
np.warnings.filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

import pandas as pd
from tqdm import tqdm

from google_drive_downloader import GoogleDriveDownloader as gdd
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.contrib.eager.python import tfe
from PIL import Image

tf.enable_eager_execution()
tf.set_random_seed(0)
np.random.seed(0)

In [0]:
gdd.download_file_from_google_drive(file_id='1ycR7Aexe8xbZ8oEDsQwGc9SIiFklRpfu', dest_path='./data.zip', unzip=True)

In [0]:
data_dir = 'data'
os.listdir(data_dir)

In [0]:
#Read data
train_df = pd.read_csv(os.path.join(data_dir, 'train.csv'))
train_df.head()

In [0]:
train_df.info()

In [0]:
test_df = pd.read_csv(os.path.join(data_dir, 'sample_submission.csv'))
test_df.head()

In [0]:
test_df.info()

In [0]:
train_df.label.value_counts()

##TODO 1: Cài đặt hàm đọc ảnh và đưa về  NumPy Array

In [0]:
#generate_data to create train and test matrix
def generate_data(image_paths, size=224):
    """
    Đọc và chuyển các ảnh về numpy array
    
    Parameters
    ----------
    image_paths: list of N strings
        List các đường dẫn ảnh
    size: int
        Kích thước ảnh cần resize
    
    Returns
    -------
    numpy array kích thước (N, size, size, 3)
    """
    image_array = np.zeros((len(image_paths), size, size, 3), dtype='float32')
    
    for idx, image_path in tqdm(enumerate(image_paths)):
        ### START CODE HERE
        image = Image.open(image_path)
        image = image.resize((size,size))
        
        # Chuyển ảnh thành numpy array và gán lại mảng image_array
        img = np.asarray(image)
        
        #normalize
        img = img/255.
        
        image_array[idx] = img
        ### END CODE HERE
    return image_array

In [0]:
# List các đường dẫn file cho việc huấn luyện
train_files = [os.path.join("data/images", file) for file in train_df.image]

# List các nhãn
train_y = train_df.label

# Tạo numpy array cho dữ liệu huấn luyện
train_arr = generate_data(train_files)

In [0]:
train_arr.shape
# train_arr[1]
# train_y.shape

In [0]:
#Tao tensor cho du lieu test

test_files = [os.path.join("data/images", file) for file in test_df.image]
test_x = generate_data(test_files)
test_x.shape

In [0]:
# test_x[:5]

In [0]:
#Tao one-hot cho train_y

num_classes = len(np.unique(train_y))
y_ohe = tf.keras.utils.to_categorical(train_y, num_classes=num_classes)

## Chia dữ liệu để huấn luyện và đánh giá

In [0]:
x_train, x_valid, y_train_ohe, y_valid_ohe = train_test_split(train_arr, y_ohe, test_size=0.25)

print("Train size: {} - Validation size: {}".format(x_train.shape, x_valid.shape))

In [0]:
# x_train[:1]

## Xây dựng mô hình

In [0]:
class ConvBlock(tf.keras.Model):
    def __init__(self, filters, kernel, strides, padding):
        '''
        Khởi tạo Convolution Block với các tham số đầu vào
        
        Parameters
        ----------
        filters: int
            số lượng filter
        kernel: int
            kích thước kernel
        strides: int
            stride của convolution layer
        padding: str
            Loại padding của convolution layer
        
        '''
        
        super(ConvBlock, self).__init__()
        ## TODO 2
        ### START CODE HERE
        
        # Tạo layer Conv2D
        self.cnn = tf.keras.layers.Conv2D(filters=filters, kernel_size=kernel, activation="relu",
                                          strides=strides, padding=padding,
                                          kernel_regularizer=tf.keras.regularizers.l2(0.0004))
        self.batchNorm = tf.keras.layers.BatchNormalization()
        
        # Tạo layer MaxPool2D
        self.pool = tf.keras.layers.AveragePooling2D(pool_size=(2,2),strides=strides, padding=padding)
        
        # Tạo các layer khác tùy ý nếu cần thiết
        
        ### END CODE HERE
        
        
    def call(self, inputs):
        '''
        Hàm này sẽ được gọi trong quá trình forwarding của mạng
        
        Parameters
        ----------
        inputs: tensor đầu vào
        
        Returns
        -------
        tensor
            giá trị đầu ra của mạng
        '''
        
        x = self.cnn(inputs)
        ## TODO 3
        ### START CODE HERE
        x = self.batchNorm(x)
        # Forward inputs qua từng layer và gán vào biến x để trả về
        
        x = self.pool(x)
        
        ## END CODE HERE

        return x

## Định nghĩa toàn bộ mô hình CNN

In [0]:
class CNN(tf.keras.Model):
    def __init__(self, num_classes):
        
        super(CNN, self).__init__()
        
        ## TODO 4
        ### START CODE HERE
        
        # Khởi tạo các convolution block
        self.block1 = ConvBlock(8, (3,3), (1,1), "valid")
        self.block2 = ConvBlock(8, (3,3), (1,1), "valid")
        self.block3 = ConvBlock(16, (3,3), (1,2), "same")
        self.block4 = ConvBlock(16, (3,3), (1,2), "same")
        self.block5 = ConvBlock(32, (3,3), (2,2), "same")
        
        self.drop = tf.keras.layers.Dropout(0.25)
        # Khởi tạo layer để flatten feature map 
        self.flatten = tf.keras.layers.Flatten()
        
        ### END CODE HERE
        
        ## TODO 5
        ### START CODE HERE
        
        # Khởi tạo fully connected layer
        self.dense1 = tf.keras.layers.Dense(400, activation='relu')
        self.dense2 = tf.keras.layers.Dense(num_classes, activation='relu')
        ### END CODE HERE

    def call(self, inputs):
        
        ## TODO 6
        
        x = self.block1(inputs)
        x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x)
        x = self.block5(x)
        
        x = self.drop(x)
     
        ### START CODE HERE
        
        # Forward gía trị inputs qua các tầng CNN và gán vào x
        
        x = self.flatten(x)
        
        ### END CODE HERE
        
        ## TODO 7
        
        ### START CODE HERE 
        
        # Forward giá trị x qua Fully connected layer
        x = self.dense1(x)
        x = self.dense2(x)
        
        ### END CODE HERE
        
        # Để sử dụng hàm softmax, ta phải thực thi trên CPU
        with tf.device('/cpu:0'):
            output = tf.nn.softmax(x)

        return output

## Huấn luyện

In [0]:
device = '/cpu:0' if tfe.num_gpus() == 0 else '/gpu:0'
batch_size = 32
epochs = 50

with tf.device(device):
    # Khởi tạo model
    model = CNN(num_classes)
    
    # Tạo callback để lưu model có accuracy trên tập validation tốt nhất
    mcp = tf.keras.callbacks.ModelCheckpoint("my_model.h5", monitor="val_acc",
                      save_best_only=True, save_weights_only=True)
    
    # Compile model
    model.compile(optimizer=tf.train.AdamOptimizer(0.001), loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    # Huấn luyện
    model.fit(x_train, y_train_ohe, batch_size=batch_size, epochs=epochs,
              validation_data=(x_valid, y_valid_ohe), verbose=1, callbacks=[mcp])


## Dự đoán các ảnh trên tập Test

### Tạo và load model đã lưu trước đó

In [0]:
# Load best model
model = CNN(num_classes)

# Thiết lập kích thước input cho model
dummy_x = tf.zeros((1, 224, 224, 3))
model._set_inputs(dummy_x)

# Load model đã lưu trước đó trong quá trình huấn luyện
model.load_weights('my_model.h5')
print("Model đã được load")

### Dự đoán nhãn của các ảnh trên tập Test

In [0]:
pred = model.predict(test_x)

# pred là một ma trận xác suất của ảnh trên các lớp.
# Ta lấy lớp có xác suất cao nhất trên từng ảnh bằng hàm argmax
pred_labels = np.argmax(pred, axis=1)

In [0]:
test_df['label'] = pred_labels
test_df.head(20)

#### Lưu kết quả thành file CSV

In [0]:
test_df.to_csv("submission.csv", index=False)